In [1]:
from transformers import BertTokenizer, EncoderDecoderModel
import torch
from datasets import load_metric

# Load the tokenizers
src_tokenizer = BertTokenizer(vocab_file='./vocab.txt')
tgt_tokenizer = BertTokenizer(vocab_file='./vocab.txt')  # Or your target language tokenizer

# Load the fine-tuned model
model = EncoderDecoderModel.from_pretrained("./results/checkpoint-1000")

# Ensure the model is in evaluation mode and move it to GPU if available
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)

EncoderDecoderModel(
  (encoder): GPT2Model(
    (wte): Embedding(357, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(357, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): Mod

In [2]:
import re

####以下函数用于讲10进制数token化#####
def format_decimal_as_hexadecimal(decimal_str):
    # Convert the decimal string to an integer
    decimal_number = int(decimal_str)
    
    # Convert the integer to a hexadecimal string
    hex_str = hex(decimal_number)[2:]  # Strip the '0x' prefix
    
    # Ensure the length of the hex string is even
    if len(hex_str) % 2 != 0:
        hex_str = '0' + hex_str
    
    # Split the hex string into pairs of characters
    hex_pairs = [hex_str[i:i+2] for i in range(0, len(hex_str), 2)]
    
    # Join the pairs with commas
    formatted_hex = ','.join(hex_pairs)
    
    return formatted_hex


def convert_number(num_str):
    num = float(num_str)
    if num.is_integer():
        num = int(num)
        sign = "-" if num < 0 else "+"
        num_str = format_decimal_as_hexadecimal(str(num).lstrip('-'))
        return f"num,{sign},{num_str},num"
    else:
        sign = "-" if num < 0 else "+"
        num_str = num_str.lstrip('-')
        integer_part, fractional_part = num_str.split('.')
        combined_num = format_decimal_as_hexadecimal(integer_part + fractional_part.rstrip('0'))
        pos_num = format_decimal_as_hexadecimal(len(fractional_part.rstrip('0')))
        return f"num,{sign},{combined_num},pos,{pos_num},num"

def process_segment(segment):
    parts = segment.split(',')
    for i, part in enumerate(parts):
        if re.match(r'^-?\d+(\.\d+)?$', part):  # Match integers and floating-point numbers
            parts[i] = convert_number(part)
    result = ','.join(parts)
    result = result.replace(",", " ")
    return result
####以上函数用于将10进制数token化：process_segment(segment)#####

####以下函数用于将payload按两位分开，用','隔开#####
def split_payload_into_pairs(text):
    # 将文本按每两个字符分割
    pairs = [text[i:i+2] for i in range(0, len(text), 2)]
    # 用逗号连接分割后的文本
    result = ' '.join(pairs)
    return result
####以上函数用于将payload按两位分开，用','隔开#####

In [3]:
def translate(src_text, input_max_length=50, output_max_length=220):
    # Tokenize the input text
    src_text = process_segment(src_text)
    inputs = src_tokenizer(src_text, return_tensors="pt", max_length=input_max_length, truncation=True, padding="max_length")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # Generate translation
    outputs = model.generate(input_ids=input_ids, 
                             attention_mask=attention_mask,
                             max_length=output_max_length,
                             num_beams=4,
                             early_stopping=True)

    # Decode the generated tokens
    payload = tgt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return payload

In [4]:
Function_Parameters = "GetAuxAxisDiagnosis,2,2," 
# Print tokenizer vocabulary size
print(f"Tokenizer vocabulary size: {len(tgt_tokenizer)}")

# Tokenize input and print token IDs
tokens = src_tokenizer(Function_Parameters, return_tensors="pt")
print(f"Input token IDs: {tokens['input_ids']}")
print(f"Max token ID: {tokens['input_ids'].max().item()}")

# Check model's embedding layer size
embedding_size = model.get_input_embeddings().num_embeddings
print(f"Model embedding size: {embedding_size}")

# Compare max token ID with embedding size
if tokens['input_ids'].max().item() >= embedding_size:
    print("Warning: Input contains token IDs outside the model's vocabulary range")

Tokenizer vocabulary size: 358
Input token IDs: tensor([[257, 313, 357, 357, 357, 357, 357, 258]])
Max token ID: 357
Model embedding size: 357


In [5]:
from datasets import load_dataset

data_files = './data_ezsocket/merged_api_para_pcapdata_dataset20k.csv'
# ; is the tab character in Python
ezsocket_dataset = load_dataset("csv", data_files=data_files, delimiter=";")
ezsocket_dataset = ezsocket_dataset.map(lambda x: {"Function and Parameters": [o.split(',', 1)[1] for o in x["Function and Parameters"]]}, batched=True) #可加速处理，删除前面的时间戳

Map:   0%|          | 0/20311 [00:00<?, ? examples/s]

In [6]:
from datasets import load_metric

fun_para = ezsocket_dataset['train']['Function and Parameters']
references = ezsocket_dataset['train']['Data Segment']
predictions_comput = []
references_comput = []
for i in range(1000):
    prediction = translate(fun_para[i])
    reference = references[i]
    predictions_comput.append([prediction])
    references_comput.append([re.sub(r"(?<=\w)(?=(?:\w\w)+$)", " ", reference)])

print(predictions_comput)
print(references_comput)
meteor = load_metric('sacrebleu')
results = meteor.compute(predictions=predictions_comput, references=references_comput)
print(results)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


[['47 49 4f 50 01 00 01 00 44 00 00 00 00 00 00 00 e0 37 00 00 01 00 1b 00 04 00 00 00 01 00 00 00 0d 00 00 00 6d 6f 63 68 61 47 65 74 44 61 74 61 00 3c 23 77 00 00 00 00 43 00 00 00 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 10 00 00 00'], ['47 49 4f 50 01 00 01 00 44 00 00 00 00 00 00 00 e0 37 00 00 01 00 1b 00 04 00 00 00 01 00 00 00 0d 00 00 00 6d 6f 63 68 61 47 65 74 44 61 74 61 00 3c 23 77 00 00 00 00 43 00 00 00 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 10 00 00 00'], ['47 49 4f 50 01 00 01 00 44 00 00 00 00 00 00 00 e0 37 00 00 01 00 1b 00 04 00 00 00 01 00 00 00 0d 00 00 00 6d 6f 63 68 61 47 65 74 44 61 74 61 00 3c 23 77 00 00 00 00 43 00 00 00 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 10 00 00 00'], ['47 49 4f 50 01 00 01 00 44 00 00 00 00 00 00 00 e0 37 00 00 01 e5 5f 01 04 00 00 00 01 00 00 00 0d 00 00 00 6d 6f 63 68 61 47 65 74 44 61 74 61 00 00 00 00 00 00 00 00 01 00 00 00 02 00 00 00 01 00 00 00 00 00 00 00 00 00 00 00 01 00 00 00'], ['47 49 4f 50 01 00

/tmp/ipykernel_399343/158615566.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric('sacrebleu')


{'score': 88.10879121818171, 'counts': [76349, 73536, 70765, 68003], 'totals': [83334, 82334, 81334, 80334], 'precisions': [91.61806705546356, 89.31425656472418, 87.00543438168539, 84.65033485199292], 'bp': 1.0, 'sys_len': 83334, 'ref_len': 80658}


In [7]:
fun_para = ezsocket_dataset['train']['Function and Parameters']
references = ezsocket_dataset['train']['Data Segment']
predictions_comput = []
references_comput = []
for i in range(1000):
    prediction = translate(fun_para[i])
    reference = references[i]
    predictions_comput.append(prediction)
    references_comput.append(re.sub(r"(?<=\w)(?=(?:\w\w)+$)", " ", reference))

meteor = load_metric('meteor')
results = meteor.compute(predictions=predictions_comput, references=references_comput)
print(results)

[nltk_data] Downloading package wordnet to /home/lizedong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/lizedong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/lizedong/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.9634210903116533}


In [8]:
fun_para = ezsocket_dataset['train']['Function and Parameters']
references = ezsocket_dataset['train']['Data Segment']
predictions_comput = []
references_comput = []
for i in range(1000):
    prediction = translate(fun_para[i])
    reference = references[i]
    predictions_comput.append(prediction)
    references_comput.append(re.sub(r"(?<=\w)(?=(?:\w\w)+$)", " ", reference))

meteor = load_metric('rouge')
results = meteor.compute(predictions=predictions_comput, references=references_comput)
print(results)

{'rouge1': AggregateScore(low=Score(precision=0.959773509092387, recall=0.9678067660725355, fmeasure=0.9636627603637886), mid=Score(precision=0.962683650749661, recall=0.9696111909882543, fmeasure=0.9659471635028238), high=Score(precision=0.9656063852580979, recall=0.971510872845187, fmeasure=0.9683351131553779)), 'rouge2': AggregateScore(low=Score(precision=0.9348254019618873, recall=0.9426303717484559, fmeasure=0.9384477136355455), mid=Score(precision=0.9396295112737149, recall=0.9460760408700752, fmeasure=0.9426600906087303), high=Score(precision=0.9434690523301636, recall=0.9491256787167683, fmeasure=0.9461190465678178)), 'rougeL': AggregateScore(low=Score(precision=0.9581187104850291, recall=0.966003812886816, fmeasure=0.9618745519502887), mid=Score(precision=0.9612813818265691, recall=0.9681515389555051, fmeasure=0.9644967691504228), high=Score(precision=0.9641879877777079, recall=0.9700320068595638, fmeasure=0.9668592317347177)), 'rougeLsum': AggregateScore(low=Score(precision=0